In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn import *
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import *
from sklearn import *
from sklearn.metrics import *
from mnist import *
from pandas import *
from datetime import date
from collections import namedtuple
import cv2
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.metrics import *
from scipy import *
from sklearn.metrics import confusion_matrix
import itertools
import os
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import * 
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.preprocessing.image import *
from keras.callbacks import *
from skimage.filters import threshold_local


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [4]:
homePath_train = "/content/gdrive/My Drive/ThucTap_T3/data/"
!ln -s "/content/gdrive/My Drive/ThucTap_T3/data" "/content/mydrive"
homePath_train = "/content/mydrive/"

In [5]:
chars = ['0', '1']
chars

['0', '1']

In [6]:
# ham xu li du lieu
def preprocessdata(homePath):
    data=[]
    label=[]
    for i,char in enumerate(chars):
        imgPath = homePath + char + "/"
        print(imgPath)
        imgFiles = os.listdir(imgPath)
#         kk = 0
        for image in imgFiles:
   
            img = cv2.imread(imgPath + image,0)
            img = cv2.resize(img, (28, 28))
            
#             print(img.shape)
#             noise_removal = cv2.bilateralFilter(img,9,75,75)

#             equal_histogram = cv2.equalizeHist(noise_removal)
#             img_pad = np.zeros([28, 28])
#             img_pad[6:26, 3:23] = img
            # print(image)
            data.append(img)
            label.append(i)

    l = len(label)
    shutfle = list(range(l))
    random.shuffle(shutfle)
    train_data = np.array(data)
    train_label = np.array(label)
    train_data = train_data[shutfle]
    train_label = train_label[shutfle]
    return data,label

In [7]:
data,label = preprocessdata(homePath_train)

/content/mydrive/0/
/content/mydrive/1/


In [8]:
x = data
y = label

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
y_tt = y_test

In [10]:
X_train = reshape(X_train,(-1,28,28,1))
y_train = to_categorical(y_train, num_classes = 2)
X_test = reshape(X_test,(-1,28,28,1))
y_test = to_categorical(y_test, num_classes = 2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: scipy.reshape is deprecated and will be removed in SciPy 2.0.0, use numpy.reshape instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
def create_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])
    
    return model

In [12]:
seed = 6
np.random.seed(seed)

In [13]:
model = KerasClassifier(build_fn = create_model, verbose = 0)

batch_size = [16, 32, 64, 128, 256, 512]
epochs = [50, 70, 100, 200, 300]

param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(X_train, y_train)

print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

# res = model.fit(X_train, y_train, validation_split=0.2,shuffle=True,
#                  batch_size=32, epochs=100, verbose=1)

# model.save("b2_t3_.h5")
# model.load_weights("b2_t3_.h5")

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 30 candidates, totalling 150 fits
[CV] batch_size=16, epochs=50 ........................................
[CV] ............ batch_size=16, epochs=50, score=0.926, total= 2.5min
[CV] batch_size=16, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.5min remaining:    0.0s


[CV] ............ batch_size=16, epochs=50, score=0.867, total= 2.4min
[CV] batch_size=16, epochs=50 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.9min remaining:    0.0s


...

[CV] ............ batch_size=16, epochs=70, score=0.932, total= 3.3min
[CV] batch_size=16, epochs=70 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 25.2min remaining:    0.0s


[CV] ............ batch_size=16, epochs=70, score=0.876, total= 3.3min
[CV] batch_size=16, epochs=100 .......................................
[CV] ........... batch_size=16, epochs=100, score=0.892, total= 4.7min
[CV] batch_size=16, epochs=100 .......................................
[CV] ........... batch_size=16, epochs=100, score=0.915, total= 4.7min
[CV] batch_size=16, epochs=100 .......................................
[CV] ........... batch_size=16, epochs=100, score=0.909, total= 4.7min
[CV] batch_size=16, epochs=100 .......................................
[CV] ........... batch_size=16, epochs=100, score=0.923, total= 4.6min
[CV] batch_size=16, epochs=100 .......................................
[CV] ........... batch_size=16, epochs=100, score=0.910, total= 4.7min
[CV] batch_size=16, epochs=200 .......................................
[CV] ........... batch_size=16, epochs=200, score=0.923, total= 9.2min
[CV] batch_size=16, epochs=200 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 415.7min finished


Best: 0.9280577182769776, using {'batch_size': 256, 'epochs': 200}
0.8878674745559693 (0.033083755171978) with: {'batch_size': 16, 'epochs': 50}
0.8835528373718262 (0.02900395847143124) with: {'batch_size': 16, 'epochs': 70}
0.9098739743232727 (0.01002381734842878) with: {'batch_size': 16, 'epochs': 100}
0.9146605849266052 (0.016282782738744978) with: {'batch_size': 16, 'epochs': 200}
0.8998238325119019 (0.021880728667138357) with: {'batch_size': 16, 'epochs': 300}
0.9074808716773987 (0.02011901576265015) with: {'batch_size': 32, 'epochs': 50}
0.9012584924697876 (0.01651968530701692) with: {'batch_size': 32, 'epochs': 70}
0.9160943269729614 (0.010310454532422698) with: {'batch_size': 32, 'epochs': 100}
0.9211987614631653 (0.008377711193629508) with: {'batch_size': 32, 'epochs': 200}
0.9234328866004944 (0.003380529594577989) with: {'batch_size': 32, 'epochs': 300}
0.911946964263916 (0.015480536593606065) with: {'batch_size': 64, 'epochs': 50}
0.9050840616226197 (0.023053322090118775) wi

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
score

AttributeError: ignored

In [ ]:
results = chars.copy()

In [ ]:
# model = Sequential()
# model = create_model()

img_ = cv2.imread("/home/giang/Pictures/anhtest3.jpg", 0)
X_test__ = cv2.resize(img_, (28, 28))

In [ ]:
y_predict = model.predict(X_test__.reshape(1,28,28,1))
test_logits = np.argmax(y_predict, axis=-1)
res__ = results[int(test_logits)]
res__

In [ ]:
kq = model.predict(X_test)
kq

In [ ]:
resS = np.argmax(kq, axis=-1)
resS

In [ ]:
print(classification_report(y_tt, resS, target_names=['0', '1']))

In [ ]:
# dau vao la duong dan cua anh (path)
# dau ra la ket qua phan loai ( co ban do hay khong )
def PrintResult(path):
    im = cv2.imread(path, 1)
    
    im_gray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)

    noise_removal = cv2.bilateralFilter(im_gray,9,75,75)

    equal_histogram = cv2.equalizeHist(noise_removal)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(5,5))
    morph_image = cv2.morphologyEx(equal_histogram,cv2.MORPH_OPEN,kernel,iterations=20)

    sub_morp_image = cv2.subtract(equal_histogram,morph_image)
    ret,thresh_image = cv2.threshold(sub_morp_image,0,255,cv2.THRESH_OTSU)

    canny_image = cv2.Canny(thresh_image,250,255)
    kernel = np.ones((3,3), np.uint8)
    dilated_image = cv2.dilate(canny_image,kernel,iterations=1)
#     plt.imshow(cv2.cvtColor(dilated_image,cv2.COLOR_BGR2RGB))
#     plt.show()
    contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contours= sorted(contours, key = cv2.contourArea, reverse = True)
    screenCnt = None
    roi_img = []
    check = 0
    for c in contours:
        peri = cv2.arcLength(c, True) 
    #     print(peri)
        if(peri < 101):
            continue

        approx = cv2.approxPolyDP(c, 0.06 * peri, True) 
        if len(approx) == 4:
            screenCnt = approx
            (x,y,w,h) = cv2.boundingRect(screenCnt)
            roi = im[y:y+h,x:x+w]
            roi_ccc = roi
            roi_img.append(roi)
    #         plt.imshow(cv2.cvtColor(roi,cv2.COLOR_BGR2RGB))
    #         plt.show()
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            X_test__ = cv2.resize(roi, (28, 28))
            y_predict = model.predict(X_test__.reshape(1,28,28,1))
    #         print(" y predict", y_predict)
            test_logits = np.argmax(y_predict, axis=-1)
            res__ = results[int(test_logits)]

            if(res__ == '1'):
                check = 1
#                 plt.imshow(cv2.cvtColor(roi_ccc,cv2.COLOR_BGR2RGB))
#                 plt.show()
   
                break

    # print(check)
    if(check == 1):
        print("anh co chua ban do")
    else:
        print("anh khong chua ban do")
        

In [ ]:
PrintResult("/home/giang/Pictures/anhtest.png")

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# !pip install mnist